In [2]:
import pandas as pd
from utils import aggregate_data, join_col_descriptions, categorize_school_level

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# df = pd.read_csv('/Users/cave/Desktop/discriminology/output/district_level_aggregates.csv', header=[0,1], index_col=0)
# df.columns = df.columns.droplevel(1)
# df.to_csv('~/Desktop/discriminology/output/db_upload_districts.csv')
# list(df.columns)

Read field mapping and data types from [dictionary](https://docs.google.com/spreadsheets/d/1mO7Tnbn9hwHcJ0hj8gBBp5MhHGDqmgGw8X3-b-s6SKs/edit#gid=0)

In [4]:
field_mappings = pd.read_excel('/Users/cave/Desktop/discriminology/field_mapping.xlsx')

### Extract methodology fields for each year

In [5]:
fields_1112 = field_mappings['11_12_field'].dropna().values
fields_1314 = field_mappings['13_14_field'].dropna().values
fields_1516 = field_mappings['15_16_field'].dropna().values

In [6]:
map_ = field_mappings[['11_12_field', '13_14_field', 'type']].dropna(subset=['11_12_field'])
map_2 = field_mappings[['11_12_field', '13_14_field']].dropna(subset=['11_12_field', '13_14_field'])
map_11_12 = pd.Series(map_2['13_14_field'].values, index=map_2['11_12_field']).to_dict()
type_dict_11_12 = pd.Series(map_['type'].values, index=map_['11_12_field']).to_dict()
final_type_dict_11_12 = {k:v for k,v in type_dict_11_12.items() if v !='str'}

In [7]:
field_mappings['col_superset'] = field_mappings['15_16_field'].combine_first(field_mappings['11_12_field'])
type_map = pd.Series(field_mappings.type.values, index=field_mappings.col_superset).to_dict()
numeric_type_map = {k:v for k,v in type_map.items() if v !='str'}

In [8]:
field_mappings[['col_superset', 'description']]
descriptions = field_mappings[['col_superset', 'description']].set_index('col_superset')

Generate SQL create statement for school table in AWS

In [11]:
create_school_table = [print(f'"{k}" double precision,') for k,v in numeric_type_map.items()]
create_school_table

"SCH_FTECOUNSELORS" double precision,
"SCH_FTESERVICES_PSY" double precision,
"SCH_FTESERVICES_SOC" double precision,
"SCH_FTESECURITY_LEO" double precision,
"SCH_FTESECURITY_GUA" double precision,
"SCH_ENR_AM_M" double precision,
"SCH_ENR_AS_M" double precision,
"SCH_ENR_HI_M" double precision,
"SCH_ENR_BL_M" double precision,
"SCH_ENR_WH_M" double precision,
"SCH_ENR_HP_M" double precision,
"SCH_ENR_TR_M" double precision,
"TOT_ENR_M" double precision,
"SCH_ENR_AM_F" double precision,
"SCH_ENR_AS_F" double precision,
"SCH_ENR_HI_F" double precision,
"SCH_ENR_BL_F" double precision,
"SCH_ENR_WH_F" double precision,
"SCH_ENR_HP_F" double precision,
"SCH_ENR_TR_F" double precision,
"TOT_ENR_F" double precision,
"SCH_GTENR_AM_M" double precision,
"SCH_GTENR_AS_M" double precision,
"SCH_GTENR_HI_M" double precision,
"SCH_GTENR_BL_M" double precision,
"SCH_GTENR_WH_M" double precision,
"SCH_GTENR_HP_M" double precision,
"SCH_GTENR_TR_M" double precision,
"TOT_GTENR_M" double precision,
"SC

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [12]:
universal_types = {'SCH_ZIP ': str
                   ,'SCHID': str
                   ,'COMBOKEY': str
                   ,'LEAID': str
                  }

### Load all data + col description files for 2011-12

In [13]:
## Reads originals from excel

# df1_1112, df1_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/05 - Overall Enrollment.xlsx', sheet_name=None).values()
# df2_1112, df2_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/08 - Students enrolled in Gifted-Talented Programs.xlsx', sheet_name=None).values()
# df3_1112, df3_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/10-1 - Students with Disabilities Served under IDEA Enrollment.xlsx', sheet_name=None).values()
# df4_1112, df4_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/10-2 - Students with Disabilities Served under 504 Enrollment.xlsx', sheet_name=None).values()
# df5_1112, df5_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/W:O Disabilities/35-3 - Students WO Disab Receiving only one out-of-school suspension.xlsx', sheet_name=None).values()
# df6_1112, df6_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/W:O Disabilities/35-4 - Students WO Disab Rec more than one out-of-school suspension.xlsx', sheet_name=None).values()
# df7_1112, df7_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/With Disabilities/36-3 - Students With Disabilities Receiving only one out-of-school suspension.xlsx', sheet_name=None).values()
# df8_1112, df8_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/With Disabilities/36-4 - Students With Disab Receiving more than one out-of-school suspension.xlsx', sheet_name=None).values()
# df9_1112, df9_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Academic/Advanced Placement/17 - Students who are taking at least one AP course.xlsx', sheet_name=None).values()
# df10_1112, df10_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Referral to law enforcement/W:O Disabilities/35-8 - Students Without Disabilities Referral to law enforcement.xlsx', sheet_name=None).values()
# df11_1112, df11_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Referral to law enforcement/With Disabilities/36-8 - Students With Disabilities Referral to law enforcement.xlsx', sheet_name=None).values()
# df12_1112, df12_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/School Related Arrest/W:O Disabilities/35-9 - Students Without Disabilities School-related arrest.xlsx', sheet_name=None).values()
# df13_1112, df13_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/School Related Arrest/With Disabilities/36-9 - Students With Disabilities School-related arrest.xlsx', sheet_name=None).values()
# df14_1112, df14_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/School Characteristics/02 - School Characteristics.xlsx', sheet_name=None).values()
# df15_1112, df15_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Staff/08-1 School Support and Security Staff (required elements).xlsx', sheet_name=None).values()
# df16_1112, df16_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/06 - Enrolled in Early Childhood and Prekindergarten.xlsx', sheet_name=None).values()

In [14]:
# ctr = 0
# for frame in desc_frames_1112:
#     frame.to_csv(f'/Users/cave/Desktop/discriminology/2011_12/descriptions/file_{ctr}.csv')
#     ctr += 1

# ctr = 0
# for frame in frames_1314:
#     frame.to_csv(f'~/Desktop/discriminology/2013_14/data/file_{ctr}.csv')
#     ctr += 1


In [15]:
df1_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_1.csv', dtype=universal_types)
df2_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_2.csv', dtype=universal_types)
df3_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_3.csv', dtype=universal_types)
df4_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_4.csv', dtype=universal_types)
df5_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_5.csv', dtype=universal_types)
df6_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_6.csv', dtype=universal_types)
df7_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_7.csv', dtype=universal_types)
df8_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_8.csv', dtype=universal_types)
df9_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_9.csv', dtype=universal_types)
df10_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_10.csv', dtype=universal_types)
df11_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_11.csv', dtype=universal_types)
df12_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_12.csv', dtype=universal_types)
df13_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_13.csv', dtype=universal_types)
df14_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_14.csv', dtype=universal_types)
df15_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_15.csv', dtype=universal_types)
df16_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_0.csv', dtype=universal_types)

/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (27,28,29,30,31,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10

In [16]:
df1_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_1.csv', index_col=0)
df2_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_2.csv', index_col=0)
df3_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_3.csv', index_col=0)
df4_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_4.csv', index_col=0)
df5_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_5.csv', index_col=0)
df6_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_6.csv', index_col=0)
df7_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_7.csv', index_col=0)
df8_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_8.csv', index_col=0)
df9_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_9.csv', index_col=0)
df10_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_10.csv', index_col=0)
df11_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_11.csv', index_col=0)
df12_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_12.csv', index_col=0)
df13_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_13.csv', index_col=0)
df14_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_14.csv', index_col=0)
df15_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_15.csv', index_col=0)
df16_desc_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/descriptions/file_0.csv', index_col=0)

In [17]:

frames_1112 = [df1_1112, df2_1112, df3_1112, df4_1112, df5_1112, df6_1112,
               df7_1112, df8_1112, df9_1112, df10_1112, df11_1112, df12_1112,
               df13_1112, df14_1112, df15_1112, df16_1112]



desc_frames_1112 = [df1_desc_1112, df2_desc_1112, df3_desc_1112, df4_desc_1112,
                    df5_desc_1112, df6_desc_1112, df7_desc_1112, df8_desc_1112,
                    df9_desc_1112, df10_desc_1112, df11_desc_1112, df12_desc_1112,
                    df13_desc_1112, df14_desc_1112, df15_desc_1112, df16_desc_1112]


Use helper function to aggregate frames and clean up negative values, standardize length of zipcodes.

In [18]:
# Careful, this cell runs for a while
df_1112_raw = aggregate_data(frames_1112, fields_1112)

In [19]:
df_1112 = df_1112_raw.copy()

In [20]:
df_1112['SCH_ZIP '] = df_1112['SCH_ZIP '].str.zfill(5) # pad zipcodes with leading zeroes

Change 0/1 indicator vars to Yes / No for selected columns

In [21]:
int_cols_to_str = [
    'PreK'
    ,'K'
    ,'G1'
    ,'G2'
    ,'G3'
    ,'G4'
    ,'G5'
    ,'G6'
    ,'G7'
    ,'G8' 
    ,'G9'
    ,'G10'
    ,'G11'
    ,'G12'
    ,'MG_SCH'
    ,'CHARTER_SCH'
    ,'ALT_SCH'
                  ]

In [22]:
df_1112[int_cols_to_str] =  df_1112[int_cols_to_str].replace('0', 'No').replace('1', 'Yes').replace(0, 'No').replace(1, 'Yes')

In [23]:
df_1112['SCH_FTESECURITY_IND'] = df_1112['SCH_FTESECURITY_IND'].str.replace('-9','No').str.replace('-5','No')
type_dict_11_12.pop('COMBOKEY')
df_1112 = df_1112.astype(final_type_dict_11_12)

Custom case handling: Juvenile Justice facilities

In [26]:
df_1112['JJ'] = df_1112['JJ'].str.replace('Z', 'Yes').str.replace('X', 'Yes')

Rename columns to match 13/14 and 15/16 data

In [27]:
df_1112.rename(columns=map_11_12, inplace=True)

In [28]:
superset = field_mappings['col_superset'].values
ordered_cols_1112 = [c for c in superset if c in df_1112.columns]

In [29]:
df_1112.index.nunique()

101133

In [30]:
df_1112.shape

(101133, 220)

In [31]:
df_1112.columns.nunique()

220

In [33]:
df_1112_final = df_1112[ordered_cols_1112]
df_1112_final['YEAR'] = '2011-12'

### Load all data + col description files for 2013-14

Collect column name descriptions from each spreadsheet

In [34]:
# # Read originals from excel

# df1_1314, df1_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/01 School Characteristics.xlsx', sheet_name=None).values()
# df2_1314, df2_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/03 Enrollment.xlsx', sheet_name=None).values()
# df3_1314, df3_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/04-1 Gifted and Talented Enrollment.xlsx', sheet_name=None).values()
# df4_1314, df4_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/06 Advanced Placement and International Baccalaureate Diploma Programme Enrollment.xlsx', sheet_name=None).values()
# df5_1314, df5_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/07-2 Advanced Placement Exams.xlsx', sheet_name=None).values()
# df6_1314, df6_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/08-1 School Support and Security Staff (required elements).xlsx', sheet_name=None).values()
# df7_1314, df7_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/09-1 Chronic Absenteeism.xlsx', sheet_name=None).values()
# df8_1314, df8_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/11-2 Suspensions (required elements).xlsx', sheet_name=None).values()
# df9_1314, df9_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/11-3 Expulsions.xlsx', sheet_name=None).values()
# df10_1314, df10_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/12 Student Referrals and Arrests.xlsx', sheet_name=None).values()
# df11_1314, df11_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/16 School Expenditures.xlsx', sheet_name=None).values()
# df12_1314, df12_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/17 Justice Facilities.xlsx', sheet_name=None).values()

In [35]:
# ctr = 0
# for frame in desc_frames_1314:
#     frame.to_csv(f'~/Desktop/discriminology/2013_14/descriptions/file_{ctr}.csv')
#     ctr += 1


# ctr = 0
# for frame in frames_1314:
#     frame.to_csv(f'~/Desktop/discriminology/2013_14/data/file_{ctr}.csv')
#     ctr += 1


In [36]:
df1_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_1.csv', dtype=universal_types)
df2_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_2.csv', dtype=universal_types)
df3_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_3.csv', dtype=universal_types)
df4_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_4.csv', dtype=universal_types)
df5_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_5.csv', dtype=universal_types)
df6_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_6.csv', dtype=universal_types)
df7_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_7.csv', dtype=universal_types)
df8_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_8.csv', dtype=universal_types)
df9_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_9.csv', dtype=universal_types)
df10_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_10.csv', dtype=universal_types)
df11_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_11.csv', dtype=universal_types)
df12_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_0.csv', dtype=universal_types)

In [37]:
df1_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_1.csv', dtype=universal_types, index_col=0)
df2_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_2.csv', dtype=universal_types, index_col=0)
df3_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_3.csv', dtype=universal_types, index_col=0)
df4_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_4.csv', dtype=universal_types, index_col=0)
df5_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_5.csv', dtype=universal_types, index_col=0)
df6_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_6.csv', dtype=universal_types, index_col=0)
df7_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_7.csv', dtype=universal_types, index_col=0)
df8_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_8.csv', dtype=universal_types, index_col=0)
df9_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_9.csv', dtype=universal_types, index_col=0)
df10_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_10.csv', dtype=universal_types, index_col=0)
df11_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_11.csv', dtype=universal_types, index_col=0)
df12_desc_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/descriptions/file_0.csv', dtype=universal_types, index_col=0)


In [38]:
frames_1314 = [df1_1314, df2_1314, df3_1314, df4_1314, df5_1314,
               df6_1314, df7_1314, df8_1314, df9_1314, df10_1314,
               df11_1314, df12_1314]


desc_frames_1314 = [df1_desc_1314, df2_desc_1314, df3_desc_1314,
                    df4_desc_1314, df5_desc_1314, df6_desc_1314,
                    df7_desc_1314, df8_desc_1314, df9_desc_1314,
                    df10_desc_1314, df11_desc_1314, df12_desc_1314]


In [39]:
df_1314_raw = aggregate_data(frames_1314, fields_1314)

In [40]:
df_1314 = df_1314_raw.copy()

In [41]:
df_1314['SCH_FTESECURITY_IND'] = df_1314['SCH_FTESECURITY_IND'].str.replace('-9','No').str.replace('-5','No')
df_1314 = df_1314.replace('-5', None)
df_1314 = df_1314.replace('-9', None)

In [42]:
df_1314.index.nunique()

95507

In [44]:
df_1314.shape

(95507, 219)

In [47]:
df_1314_final = df_1314
df_1314_final['YEAR'] = '2013-14'

### Isolate numeric columns in 15/16 and replace negative numbers with zeroes.

In [49]:
df1516_raw = pd.read_csv('/Users/cave/Desktop/discriminology/OCR School data sample/2015:16/CRDC 2015-16 School Data copy.csv'
                      , encoding='iso-8859-1'
                     )
df1516_raw['LEAID'] = df1516_raw['LEAID'].astype(str).str.zfill(7)
df1516_raw['SCHID'] = df1516_raw['SCHID'].astype(str).str.zfill(5)
df1516_raw['COMBOKEY'] = df1516_raw['LEAID'] + df1516_raw['SCHID']
df1516_raw = df1516_raw[fields_1516]

/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,6,23,24,25,30,31,1828) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Replace negative values with zeroes.

In [50]:
num = df1516_raw._get_numeric_data()
num[num < 0] = 0

In [51]:
df1516_raw = df1516_raw.replace('-5', 'No').replace('-9', 'No')

In [52]:
df_1516_final = df1516_raw.set_index('COMBOKEY')
df_1516_final['YEAR'] = '2015-16'

### Concatenate all three years of data together

In [55]:
print('11/12 Data Summary:')
districts_11_12 = df_1112_final['LEAID'].nunique()
schools_11_12 = df_1112_final.index.nunique()
print(f"Unique districts: {districts_11_12}")
print(f"Unique schools: {schools_11_12}")

11/12 Data Summary:
Unique districts: 17342
Unique schools: 101133


In [56]:
print('13/14 Data Summary:')
districts_13_14 = df_1314_final['LEAID'].nunique()
schools_13_14 = df_1314_final.index.nunique()
print(f"Unique districts: {districts_13_14}")
print(f"Unique schools: {schools_13_14}")

13/14 Data Summary:
Unique districts: 16758
Unique schools: 95507


In [57]:
print('15/16 Data Summary:')
districts_1516 = df_1516_final['LEAID'].nunique()
schools_1516 = df_1516_final.index.nunique()
print(f"Unique districts: {districts_1516}")
print(f"Unique schools: {schools_1516}")

15/16 Data Summary:
Unique districts: 17337
Unique schools: 96360


In [74]:
full_table = pd.concat([df_1516_final, df_1314_final, df_1112_final], axis=0)
full_table = full_table.astype(numeric_type_map)
full_table.sort_index(inplace=True)
full_table.reset_index(inplace=True)
full_table['SCHID'] = full_table['SCHID'].astype(str).str.zfill(5)

Impute missing district state names from other years 

In [84]:
# Fill in LEA STATE with non-null values in LEAID group
full_table['LEA_STATE'] = full_table.groupby(['LEAID'])['LEA_STATE'].fillna(method='ffill').fillna(method='bfill')
full_table['LEA_STATE_NAME'] = full_table.groupby(['LEAID'])['LEA_STATE_NAME'].fillna(method='ffill').fillna(method='bfill')
full_table['LEA_NAME'] = full_table.groupby(['LEAID'])['LEA_NAME'].fillna(method='ffill').fillna(method='bfill')



In [85]:
full_table['SCH_NAME'] = full_table.groupby(['SCHID'])['SCH_NAME'].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_ZIP '] = full_table.groupby(['SCHID'])['SCH_ZIP '].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_CITY'] = full_table.groupby(['SCHID'])['SCH_CITY'].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_ADDRESS'] = full_table.groupby(['SCHID'])['SCH_ADDRESS'].fillna(method='ffill').fillna(method='bfill')

Capitalize the first letter in each district name, school name, address.

In [86]:
full_table['LEA_NAME'] = full_table['LEA_NAME'].str.title()
full_table['SCH_NAME'] = full_table['SCH_NAME'].str.title()
full_table['SCH_ADDRESS'] = full_table['SCH_ADDRESS'].str.title()
full_table['SCH_CITY'] = full_table['SCH_CITY'].str.title()

Categorize each school as elementary, middle, high, or other using grade enrollment data.

In [91]:
full_table['SCH_LEVEL'] = full_table.apply(lambda row: categorize_school_level(row['SCH_GRADE_KG'], row['SCH_GRADE_G01'],
                            row['SCH_GRADE_G02'], row['SCH_GRADE_G03'], row['SCH_GRADE_G04'], row['SCH_GRADE_G05'],
                            row['SCH_GRADE_G06'], row['SCH_GRADE_G07'], row['SCH_GRADE_G08'], row['SCH_GRADE_G09'],
                            row['SCH_GRADE_G10'], row['SCH_GRADE_G11'], row['SCH_GRADE_G12']), axis=1
                            )

In [93]:
full_table[['SCH_LEVEL', 'SCH_GRADE_KG', 'SCH_GRADE_G01',
            'SCH_GRADE_G02', 'SCH_GRADE_G03', 'SCH_GRADE_G04', 'SCH_GRADE_G05',
            'SCH_GRADE_G06', 'SCH_GRADE_G07', 'SCH_GRADE_G08', 'SCH_GRADE_G09',
            'SCH_GRADE_G10', 'SCH_GRADE_G11', 'SCH_GRADE_G12']]

,SCH_LEVEL,SCH_GRADE_KG,SCH_GRADE_G01,SCH_GRADE_G02,SCH_GRADE_G03,SCH_GRADE_G04,SCH_GRADE_G05,SCH_GRADE_G06,SCH_GRADE_G07,SCH_GRADE_G08,SCH_GRADE_G09,SCH_GRADE_G10,SCH_GRADE_G11,SCH_GRADE_G12
0,Secondary School,No,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes
1,Secondary School,No,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes
2,Secondary School,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
3,Secondary School,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,Secondary School,No,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292995,High School,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes
292996,Other,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
292997,Secondary School,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes
292998,High School,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes


In [97]:
full_table.to_csv('/Users/cave/Desktop/discriminology/output/db_upload_schools.csv')

In [ ]:
full_table_final = join_col_descriptions(full_table)

In [ ]:
full_table_final.to_csv('/Users/cave/Desktop/discriminology/output/full_table_all_years.csv')
full_table_final[:2000].to_csv('/Users/cave/Desktop/discriminology/output/full_table_sample.csv')

In [ ]:
full_table_final.shape

In [ ]:
full_table_final.head(50)

### Group by districts and sum over fields

In [ ]:
# full_table = pd.read_csv('/Users/cave/Desktop/discriminology/output/full_table_all_years.csv', header=[0,1], index_col=0)

In [98]:
district = full_table.copy()

In [100]:
# district.columns = district.columns.droplevel(1)
district = district.astype(numeric_type_map)

In [101]:
district['TOTAL_SCHOOLS'] = 1.0
district['TOTAL_ENROLLMENT'] = district['TOT_ENR_F'] + district['TOT_ENR_M']

Custom aggregation dictionary - for string values, take the first non-null value.  For numeric fields, take the sum

In [102]:
agg_cols = ['LEA_NAME', 'LEA_STATE_NAME', 'TOTAL_SCHOOLS', 'TOTAL_ENROLLMENT',
'SCH_DISCWODIS_MULTOOS_HI_M',
 'SCH_DISCWDIS_ARR_IDEA_AM_M',
 'SCH_GTENR_IDEA_F',
 'SCH_DISCWODIS_SINGOOS_HP_F',
 'SCH_FTECOUNSELORS',
 'SCH_DISCWODIS_ARR_WH_M',
 'TOT_DISCWDIS_ARR_IDEA_M',
 'SCH_GTENR_TR_M',
 'SCH_DISCWODIS_MULTOOS_AS_M',
 'SCH_APENR_AM_F',
 'SCH_DISCWODIS_REF_BL_F',
 'SCH_DISCWODIS_ARR_AM_F',
 'TOT_DISCWODIS_REF_M',
 'SCH_IDEAENR_HI_M',
 'SCH_FTESERVICES_PSY',
 'SCH_DISCWDIS_MULTOOS_IDEA_BL_F',
 'TOT_APENR_F',
 'SCH_DISCWODIS_ARR_WH_F',
 'SCH_IDEAENR_HP_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_TR_F',
 'SCH_DISCWDIS_REF_IDEA_WH_F',
 'SCH_DISCWODIS_MULTOOS_WH_M',
 'SCH_FTESECURITY_GUA',
 'SCH_DISCWODIS_SINGOOS_TR_M',
 'SCH_DISCWDIS_REF_IDEA_WH_M',
 'SCH_DISCWODIS_ARR_TR_M',
 'SCH_DISCWDIS_ARR_IDEA_BL_F',
 'SCH_DISCWDIS_ARR_IDEA_TR_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_BL_M',
 'SCH_DISCWODIS_ARR_AM_M',
 'SCH_DISCWDIS_ARR_IDEA_AS_M',
 'SCH_DISCWDIS_REF_IDEA_BL_M',
 'SCH_GTENR_IDEA_M',
 'TOT_DISCWODIS_MULTOOS_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_HI_F',
 'TOT_DISCWODIS_SINGOOS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AM_F',
 'SCH_DISCWODIS_MULTOOS_TR_F',
 'TOT_DISCWODIS_SINGOOS_M',
 'SCH_ENR_HI_M',
 'SCH_APENR_HI_F',
 'SCH_IDEAENR_HI_F',
 'SCH_DISCWDIS_REF_IDEA_BL_F',
 'SCH_DISCWODIS_MULTOOS_AM_F',
 'TOT_DISCWDIS_MULTOOS_IDEA_M',
 'SCH_DISCWDIS_REF_IDEA_AS_M',
 'SCH_GTENR_HI_M',
 'SCH_DISCWODIS_REF_HI_M',
 'SCH_FTESERVICES_SOC',
 'SCH_DISCWDIS_SINGOOS_IDEA_AM_F',
 'SCH_DISCWODIS_REF_TR_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_HP_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_BL_M',
 'TOT_ENR_F',
 'SCH_GTENR_AS_F',
 'SCH_GTENR_HP_F',
 'SCH_ENR_BL_F',
 'SCH_DISCWODIS_SINGOOS_BL_M',
 'SCH_DISCWODIS_SINGOOS_TR_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_WH_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AS_F',
 'SCH_DISCWODIS_SINGOOS_HP_M',
 'SCH_GTENR_WH_M',
 'SCH_DISCWODIS_REF_AM_F',
 'SCH_DISCWODIS_SINGOOS_AS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_HI_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HI_M',
 'SCH_DISCWODIS_SINGOOS_HI_M',
 'SCH_APENR_TR_M',
 'SCH_DISCWODIS_REF_HI_F',
 'SCH_DISCWODIS_ARR_BL_F',
 'SCH_ENR_TR_F',
 'SCH_DISCWODIS_SINGOOS_BL_F',
 'SCH_DISCWODIS_ARR_HP_F',
 'SCH_APENR_AS_M',
 'SCH_GTENR_LEP_M',
 'SCH_DISCWDIS_REF_IDEA_HI_F',
 'SCH_DISCWODIS_MULTOOS_BL_F',
 'SCH_DISCWODIS_REF_HP_M',
 'SCH_APENR_BL_F',
 'SCH_IDEAENR_AS_M',
 'TOT_DISCWODIS_ARR_M',
 'SCH_DISCWODIS_SINGOOS_HI_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_WH_F',
 'SCH_IDEAENR_AM_F',
 'TOT_APENR_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_TR_M',
 'SCH_DISCWODIS_REF_TR_F',
 'SCH_DISCWDIS_REF_IDEA_AM_M',
 'SCH_GTENR_BL_F',
 'SCH_IDEAENR_TR_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AM_M',
 'SCH_GTENR_HI_F',
 'SCH_DISCWODIS_ARR_AS_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_TR_F',
 'TOT_DISCWODIS_MULTOOS_F',
 'SCH_DISCWODIS_REF_AS_M',
 'SCH_DISCWDIS_ARR_IDEA_WH_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_BL_F',
 'TOT_DISCWDIS_SINGOOS_IDEA_M',
 'SCH_APENR_HI_M',
 'SCH_DISCWODIS_SINGOOS_WH_F',
 'SCH_DISCWDIS_ARR_IDEA_HP_M',
 'SCH_IDEAENR_AM_M',
 'SCH_APENR_AM_M',
 'SCH_GTENR_TR_F',
 'TOT_GTENR_F',
 'SCH_DISCWODIS_SINGOOS_AM_F',
 'SCH_APENR_WH_M',
 'SCH_DISCWODIS_REF_AM_M',
 'TOT_DISCWODIS_REF_F',
 'SCH_IDEAENR_BL_F',
 'SCH_DISCWODIS_SINGOOS_AS_M',
 'TOT_DISCWDIS_SINGOOS_IDEA_F',
 'SCH_DISCWODIS_ARR_HI_M',
 'SCH_DISCWDIS_ARR_IDEA_HP_F',
 'SCH_DISCWDIS_REF_IDEA_HI_M',
 'SCH_ENR_TR_M',
 'SCH_GTENR_BL_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_AM_M',
 'SCH_DISCWODIS_ARR_HI_F',
 'SCH_IDEAENR_AS_F',
 'SCH_DISCWDIS_ARR_IDEA_HI_F',
 'SCH_APENR_HP_F',
 'TOT_IDEAENR_M',
 'SCH_GTENR_AM_M',
 'SCH_ENR_BL_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_AS_M',
 'SCH_ENR_AM_M',
 'SCH_DISCWODIS_MULTOOS_WH_F',
 'SCH_ENR_HP_M',
 'SCH_IDEAENR_BL_M',
 'SCH_IDEAENR_TR_M',
 'SCH_APENR_WH_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_AS_F',
 'SCH_ENR_AM_F',
 'SCH_DISCWDIS_REF_IDEA_TR_M',
 'SCH_DISCWODIS_REF_AS_F',
 'SCH_DISCWDIS_ARR_IDEA_AM_F',
 'SCH_ENR_WH_M',
 'SCH_DISCWODIS_REF_BL_M',
 'SCH_DISCWODIS_MULTOOS_AM_M',
 'SCH_DISCWDIS_ARR_IDEA_HI_M',
 'SCH_DISCWODIS_REF_HP_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_AS_M',
 'SCH_DISCWODIS_ARR_AS_M',
 'SCH_IDEAENR_WH_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_HP_F',
 'TOT_DISCWDIS_REF_IDEA_M',
 'SCH_FTESECURITY_LEO',
 'SCH_DISCWODIS_MULTOOS_TR_M',
 'SCH_IDEAENR_HP_F',
 'SCH_DISCWODIS_MULTOOS_HI_F',
 'SCH_DISCWDIS_ARR_IDEA_WH_F',
 'SCH_GTENR_HP_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_TR_M',
 'SCH_DISCWDIS_ARR_IDEA_AS_F',
 'SCH_DISCWODIS_REF_WH_M',
 'SCH_DISCWODIS_ARR_BL_M',
 'TOT_IDEAENR_F',
 'TOT_DISCWDIS_ARR_IDEA_F',
 'SCH_DISCWODIS_MULTOOS_AS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_WH_M',
 'SCH_APENR_TR_F',
 'SCH_ENR_WH_F',
 'SCH_DISCWODIS_REF_WH_F',
 'SCH_APENR_HP_M',
 'SCH_DISCWODIS_MULTOOS_BL_M',
 'SCH_GTENR_AM_F',
 'TOT_DISCWDIS_MULTOOS_IDEA_F',
 'SCH_DISCWDIS_REF_IDEA_HP_M',
 'SCH_ENR_HI_F',
 'SCH_DISCWDIS_REF_IDEA_HP_F',
 'TOT_ENR_M',
 'SCH_ENR_AS_F',
 'SCH_DISCWDIS_REF_IDEA_AM_F',
 'SCH_DISCWODIS_ARR_HP_M',
 'SCH_APENR_AS_F',
 'TOT_DISCWODIS_ARR_F',
 'SCH_ENR_AS_M',
 'SCH_GTENR_WH_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HP_F',
 'SCH_GTENR_LEP_F',
 'SCH_IDEAENR_WH_F',
 'SCH_DISCWODIS_SINGOOS_AM_M',
 'SCH_DISCWODIS_SINGOOS_WH_M',
 'SCH_DISCWDIS_ARR_IDEA_TR_F',
 'SCH_ENR_HP_F',
 'SCH_DISCWODIS_MULTOOS_HP_M',
 'SCH_DISCWDIS_REF_IDEA_TR_F',
 'SCH_GTENR_AS_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_WH_M',
 'TOT_GTENR_M',
 'SCH_DISCWODIS_ARR_TR_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HP_M',
 'SCH_DISCWODIS_MULTOOS_HP_F',
 'SCH_DISCWDIS_ARR_IDEA_BL_M',
 'SCH_DISCWDIS_REF_IDEA_AS_F',
 'SCH_APENR_BL_M',
 'TOT_DISCWDIS_REF_IDEA_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_HI_M']

In [103]:
agg_dict = {k:sum if k not in {'LEA_NAME', 'LEA_STATE_NAME'} else 'first' for k in agg_cols}

In [106]:
len(agg_dict) == len(agg_cols)

True

In [107]:
grouped_by_distyr = district.groupby(['LEAID', 'YEAR'])[agg_cols].agg(agg_dict)
grouped_by_distyr = grouped_by_distyr[agg_cols]
grouped_by_distyr.reset_index(inplace=True)
grouped_by_distyr.sort_values(by=['LEAID', 'YEAR'], inplace=True)

Read geodata from file and join to district aggregation.

In [112]:
sch_geo_1516 = pd.read_excel('/Users/cave/Desktop/discriminology/Geocoded Schools:Districts/2015:16/Schools_EDGE_GEOCODE_PUBLICSCH_1516.xlsx', dtype={'NCESSCH': str})
dist_geo_1516 = pd.read_excel('/Users/cave/Desktop/discriminology/Geocoded Schools:Districts/2015:16/Districts_EDGE_GEOCODE_PUBLICLEA_1516.xlsx', dtype={'LEAID': str})


In [113]:
sch_geo_1516['NCESSCH'] = sch_geo_1516['NCESSCH'].str.zfill(12)
dist_geo_1516['LEAID'] = dist_geo_1516['LEAID'].str.zfill(7)
sch_geo_1516.NCESSCH.nunique()

102209

In [114]:
sch_geo_1819 = pd.read_excel('/Users/cave/Desktop/discriminology/Geocoded Schools:Districts/2018:19/Schools_EDGE_GEOCODE_PUBLICSCH_1819.xlsx', dtype={'NCESSCH': str})
dist_geo_1819 = pd.read_excel('/Users/cave/Desktop/discriminology/Geocoded Schools:Districts/2018:19/Districts_EDGE_GEOCODE_PUBLICLEA_1819.xlsx', dtype={'LEAID': str})

In [115]:
sch_geo_1819['NCESSCH'] = sch_geo_1819['NCESSCH'].str.zfill(12)
dist_geo_1819['LEAID'] = dist_geo_1819['LEAID'].str.zfill(7)
sch_geo_1819.NCESSCH.nunique()

102176

In [116]:
geo_data = pd.read_excel('/Users/cave/Desktop/discriminology/LEA Profile Info.xlsx', dtype={'LEAID': str})
geo_data['LEAID'] = geo_data['LEAID'].str.zfill(7)

In [117]:
len(geo_data)

16758

In [119]:
len(set(grouped_by_distyr['LEAID'].values))

18135

In [120]:
len(set(geo_data['LEAID'].values).intersection(set(grouped_by_distyr['LEAID'].values)))

16758

In [121]:
district_geo = geo_data[['LEAID', 'LEA_ADDRESS', 'LEA_CITY', 'LEA_ZIP',
       'CJJ', 'LEA_ENR', 'LEA_SCHOOLS', 'Latitude', 'Longitude']]

In [122]:
district_geo.head()

,LEAID,LEA_ADDRESS,LEA_CITY,LEA_ZIP,CJJ,LEA_ENR,LEA_SCHOOLS,Latitude,Longitude
0,0100002,1000 INDUSTRIAL SCHOOL ROAD,MT. MEIGS,36057,Yes,3674,3,32.371901,-86.083791
1,0100005,107 WEST MAIN STREET,ALBERTVILLE,35950,No,4712,6,34.268415,-86.209216
2,0100006,12380 US HIGHWAY 431 S,GUNTERSVILLE,35976,No,5624,14,34.305366,-86.287334
3,0100007,2810 METROPOLITAN WAY,HOOVER,35243,No,14054,16,33.445932,-86.750203
4,0100008,211 CELTIC DRIVE,MADISON,35758,Yes,9206,11,34.686749,-86.745762


In [123]:
district_group_w_geo = pd.merge(grouped_by_distyr, district_geo, left_on='LEAID', right_on='LEAID', how='left')

In [124]:
district_group_w_geo.head()

,LEAID,YEAR,LEA_NAME,LEA_STATE_NAME,TOTAL_SCHOOLS,TOTAL_ENROLLMENT,SCH_DISCWODIS_MULTOOS_HI_M,SCH_DISCWDIS_ARR_IDEA_AM_M,SCH_GTENR_IDEA_F,SCH_DISCWODIS_SINGOOS_HP_F,...,TOT_DISCWDIS_REF_IDEA_F,SCH_DISCWDIS_MULTOOS_IDEA_HI_M,LEA_ADDRESS,LEA_CITY,LEA_ZIP,CJJ,LEA_ENR,LEA_SCHOOLS,Latitude,Longitude
0,0100002,2011-12,Alabama Youth Services,ALABAMA,4.0,1098.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1000 INDUSTRIAL SCHOOL ROAD,MT. MEIGS,36057.0,Yes,3674.0,3.0,32.371901,-86.083791
1,0100002,2013-14,Alabama Youth Services,ALABAMA,3.0,3702.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1000 INDUSTRIAL SCHOOL ROAD,MT. MEIGS,36057.0,Yes,3674.0,3.0,32.371901,-86.083791
2,0100002,2015-16,Alabama Youth Services,ALABAMA,4.0,1126.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1000 INDUSTRIAL SCHOOL ROAD,MT. MEIGS,36057.0,Yes,3674.0,3.0,32.371901,-86.083791
3,0100005,2011-12,Albertville City,ALABAMA,6.0,4149.0,8.0,0.0,0.0,0.0,...,2.0,2.0,107 WEST MAIN STREET,ALBERTVILLE,35950.0,No,4712.0,6.0,34.268415,-86.209216
4,0100005,2013-14,Albertville City,ALABAMA,6.0,4723.0,9.0,0.0,0.0,0.0,...,0.0,4.0,107 WEST MAIN STREET,ALBERTVILLE,35950.0,No,4712.0,6.0,34.268415,-86.209216


In [132]:
text_cols = ['LEAID', 'LEA_NAME', "YEAR", "LEA_STATE", "LEA_STATE_NAME",
    "LEAID", "LEA_NAME", 'LEA_ADDRESS', 'LEA_CITY', 'CJJ',
    ]

In [138]:
district_create = [print(f'"{k}" double precision,') if k not in text_cols else print(f'"{k}" text,') for k in district_group_w_geo.columns]

"LEAID" text,
"YEAR" text,
"LEA_NAME" text,
"LEA_STATE_NAME" text,
"TOTAL_SCHOOLS" double precision,
"TOTAL_ENROLLMENT" double precision,
"SCH_DISCWODIS_MULTOOS_HI_M" double precision,
"SCH_DISCWDIS_ARR_IDEA_AM_M" double precision,
"SCH_GTENR_IDEA_F" double precision,
"SCH_DISCWODIS_SINGOOS_HP_F" double precision,
"SCH_FTECOUNSELORS" double precision,
"SCH_DISCWODIS_ARR_WH_M" double precision,
"TOT_DISCWDIS_ARR_IDEA_M" double precision,
"SCH_GTENR_TR_M" double precision,
"SCH_DISCWODIS_MULTOOS_AS_M" double precision,
"SCH_APENR_AM_F" double precision,
"SCH_DISCWODIS_REF_BL_F" double precision,
"SCH_DISCWODIS_ARR_AM_F" double precision,
"TOT_DISCWODIS_REF_M" double precision,
"SCH_IDEAENR_HI_M" double precision,
"SCH_FTESERVICES_PSY" double precision,
"SCH_DISCWDIS_MULTOOS_IDEA_BL_F" double precision,
"TOT_APENR_F" double precision,
"SCH_DISCWODIS_ARR_WH_F" double precision,
"SCH_IDEAENR_HP_M" double precision,
"SCH_DISCWDIS_MULTOOS_IDEA_TR_F" double precision,
"SCH_DISCWDIS_REF_IDEA_WH

In [144]:
dist_type_map = {k:v for k,v in type_map.items() if k in district_group_w_geo.columns}

In [146]:
district_group_w_geo = district_group_w_geo.astype(dist_type_map)

In [147]:
district_group_w_geo.to_csv('/Users/cave/Desktop/discriminology/output/db_upload_districts.csv')

In [ ]:
district_merge = pd.merge(district_group_w_geo.T, descriptions, left_index=True, right_index=True, how='left').set_index('description', append=True).T

In [ ]:
sample = district_merge[~district_merge['Latitude', None].isna()]
sample[-600:].to_csv('~/Desktop/discriminology/output/district_level_sample.csv')

### Code Sandbox - everything below only needs to be run once

In [ ]:
# descriptions = pd.concat(desc_frames_1112, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1112]
# descriptions.columns = ['2011_12_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2011_12_field_descriptions.csv')

# descriptions = pd.concat(desc_frames_1314, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1314]
# descriptions.columns = ['2013_14_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2013_14_field_descriptions.csv')